In [1]:
import sys
sys.path.append("..")
sys.path.append("../src/")
sys.path.append("../src/algs/")

from src import interface
from src import simulator
from src import algs

import numpy as np
import matplotlib.pyplot as plt

# Experiment 1: Greedy Vs Deferred Vs Lookahead
-- batch saved for later

In [2]:
def weight_function_distance(buyer_pos, buyer_d, seller_pos, seller_d):
    bx, by = buyer_pos
    sx, sy = seller_pos
    return 1/(1 + (bx-sx)**2 + (by-sy)**2)

In [3]:
departure_distr = (4,1) # mean, var
seed = 42
p_node = 1 # add a node every step
num_runs = 1000

In [4]:
algs_to_test = [
    ("greedy", algs.Greedy()),
    ("dfa", algs.DynamicDeferredAcceptance()),
    ("lookahead1", algs.DeferredWithLookAhead(1)),
    ("lookahead3", algs.DeferredWithLookAhead(3)),
    ("lookahead5", algs.DeferredWithLookAhead(5))
]

## actually run one of the experiments

In [5]:
results = {}
for (alg_name, alg) in algs_to_test:
    inter = interface.Interface(
        alg,
        simulator.Simulator(weight_function_distance),
        p_node=p_node,
        seed=seed,
        dep_distr=departure_distr,
        verbose=False)
    results[alg_name] = inter.run(num_runs)

In [6]:
for alg_name, res in results.items():
    average_weight =  np.mean([w[0] for w in res[0]])
    print("The {} algorithm had an average weight of {}".format(alg_name, average_weight))


The greedy algorithm had an average weight of 0.17258985526348877
The dfa algorithm had an average weight of 0.15539110240281198
The lookahead1 algorithm had an average weight of 0.15539110240281198
The lookahead3 algorithm had an average weight of 0.15539110240281198
The lookahead5 algorithm had an average weight of 0.15539110240281198


# Let's run a few experiments...

In [7]:
def run_an_experiment(seed, p_node, departure_distr, algs_to_test):
    results = {}
    for (alg_name, alg) in algs_to_test:
        inter = interface.Interface(
            alg,
            simulator.Simulator(weight_function_distance),
            p_node=p_node,
            seed=seed,
            dep_distr=departure_distr)
        results[alg_name] = inter.run(num_runs)
    for alg_name, res in results.items():
        average_weight =  np.mean([w[0] for w in res[0]])
        print("The {} algorithm had an average weight of {}".format(alg_name, average_weight))



In [8]:
for seed in range(0,10):
    print("\nSeed:", seed)
    run_an_experiment(seed, 1, (4,1), algs_to_test[:3])


Seed: 0
The greedy algorithm had an average weight of 0.19492817795521916
The dfa algorithm had an average weight of 0.15928865410333548
The lookahead1 algorithm had an average weight of 0.14659745905134505

Seed: 1
The greedy algorithm had an average weight of 0.1938486558698566
The dfa algorithm had an average weight of 0.17409650788278988
The lookahead1 algorithm had an average weight of 0.17409650788278988

Seed: 2
The greedy algorithm had an average weight of 0.15230711133930055
The dfa algorithm had an average weight of 0.13558792113115756
The lookahead1 algorithm had an average weight of 0.13558792113115756

Seed: 3
The greedy algorithm had an average weight of 0.18638928322634538
The dfa algorithm had an average weight of 0.16896687373126087
The lookahead1 algorithm had an average weight of 0.16896687373126087

Seed: 4
The greedy algorithm had an average weight of 0.17851957915978198
The dfa algorithm had an average weight of 0.163213977327564
The lookahead1 algorithm had an a